In [1]:
require 'nn'
require 'rnn'

In [2]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

function buildPredsummary(summary, chosenactions, inputsentences, select_index)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    for i=1, chosenactions:size(1) do
        -- the 2 is for the SELECT index, will have to make this more general later
        if chosenactions[i][select_index] == 1 then
            summary[i]:copy(inputsentences[i])
        end
    end    
    return summary
end

function buildPredsummaryFast(summary, chosenactions, inputsentences, select_index)
    n = inputsentences:size(1)
    k = inputsentences:size(2)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    actionmatrix = chosenactions:select(2, select_index):clone():resize(n, 1):view(n, 1):expand(n, k):clone()
--     actionmatrix = chosenactions:select(2, select_index):resize(1, n):view(n, 1):expand(n, k):clone()
    return actionmatrix:cmul(inputsentences:double())
end

In [3]:
-- Setting parameters
n = 10
n_s = 5
k = 7
q = 5
a = 1
b = 100
embDim = 50
SKIP = 1
SELECT = 2

In [4]:
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)
    
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)
    
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

In [5]:
local SKIP = 1
local SELECT = 2

-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)
    
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)
    
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

model = buildModel('bow', b, embDim, 'f1', false, false)
preds = model:forward({sentences[1], queries, torch.zeros(n, q)})
print("predictions = ")
print(preds)

-- Pulling the best actions
qMax, qindx = torch.max(preds, 2)

-- Here's the fast way to select the optimal action for each query
actions = torch.zeros(n, 2):scatter(2, qindx, torch.ones(preds:size()))

Running bag-of-words model to learn f1	


predictions = 	
 0.0162  0.0237
 0.0085  0.1230
-0.1445 -0.1328
 0.1654 -0.0489
 0.1633 -0.2186
-0.1485  0.0486
 0.0533 -0.2945
-0.0102  0.0866
 0.1421  0.0050
-0.2705 -0.1776
[torch.DoubleTensor of size 10x2]



In [86]:
totalPredsummary = torch.zeros(n, 2 * k)
predsummary1 = buildPredsummaryFast(predsummary, actions, sentences[1], SELECT)
predsummary2 = buildPredsummaryFast(predsummary, actions, sentences[2], SELECT)

In [87]:
predsummary1, predsummary2

 12  36   6  83  58  39  76
 48   6   2  27  30   7  48
 36  40  12  37   5  24  27
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 70  24   1  88  70  34  20
  0   0   0   0   0   0   0
  2  85  17  51  90  34  55
  0   0   0   0   0   0   0
 97  50  28  46   9  64  21
[torch.DoubleTensor of size 10x7]

 13   9  54  80  39  26  91
 20  84  73  20  74  68  47
 77  29  66  97   5  38  74
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 84  25  77  64  98  41  26
  0   0   0   0   0   0   0
 67  44  87  73  50  93  53
  0   0   0   0   0   0   0
 97  83  91  78  48  56  36
[torch.DoubleTensor of size 10x7]



In [88]:
totalPredsummary

 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
[torch.DoubleTensor of size 10x14]



In [33]:
--- [predsummary1:eq(0)]
predsummary1 

 12  36   6  83  58  39  76
 48   6   2  27  30   7  48
 36  40  12  37   5  24  27
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 70  24   1  88  70  34  20
  0   0   0   0   0   0   0
  2  85  17  51  90  34  55
  0   0   0   0   0   0   0
 97  50  28  46   9  64  21
[torch.DoubleTensor of size 10x7]



In [36]:
predsummary1[1]

 12
 36
  6
 83
 58
 39
 76
[torch.DoubleTensor of size 7]



In [40]:
totalPredsummary[1]:resize(predsummary[1]:size(1)):copy(predsummary[1])

 12
 36
  6
 83
 58
 39
 76
[torch.DoubleTensor of size 7]



In [64]:
totalPredsummary[1][{{14-7+1, 14}}]:copy(predsummary1[1])

 12
 36
  6
 83
 58
 39
 76
[torch.DoubleTensor of size 7]



In [67]:
predsummary1[1]

 12
 36
  6
 83
 58
 39
 76
[torch.DoubleTensor of size 7]



In [78]:
predsummary1

 12  36   6  83  58  39  76
 48   6   2  27  30   7  48
 36  40  12  37   5  24  27
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 70  24   1  88  70  34  20
  0   0   0   0   0   0   0
  2  85  17  51  90  34  55
  0   0   0   0   0   0   0
 97  50  28  46   9  64  21
[torch.DoubleTensor of size 10x7]



In [90]:
totalPredsummary:copy(predsummary1)

[string "local f = function() return totalPredsummary:..."]:1: inconsistent tensor size at /Users/franciscojavierarceo/torch/pkg/torch/lib/TH/generic/THTensorCopy.c:7
stack traceback:
	[C]: in function 'f'
	[string "local f = function() return totalPredsummary:..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010ef62d20: 

In [96]:
predsummary1

 12  36   6  83  58  39  76
 48   6   2  27  30   7  48
 36  40  12  37   5  24  27
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 70  24   1  88  70  34  20
  0   0   0   0   0   0   0
  2  85  17  51  90  34  55
  0   0   0   0   0   0   0
 97  50  28  46   9  64  21
[torch.DoubleTensor of size 10x7]



In [113]:
totalPredsummary:index(1, torch.LongTensor({1, 3}))

 0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0
[torch.DoubleTensor of size 2x14]



In [116]:
-- totalPredsummary:index(1, torch.LongTensor({1, 3}))
-- :copy(predsummary1:index(1, torch.LongTensor({1, 3})))

In [117]:
totalPredsummary[{{1, 2}, {2, 3}}], predsummary1[{{1, 3}, {1, 3 }}]

 0  0
 0  0
[torch.DoubleTensor of size 2x2]

 12  36   6
 48   6   2
 36  40  12
[torch.DoubleTensor of size 3x3]



In [83]:
totalPredsummary:resize(predsummary1:size()):copy(predsummary1)

 12  36   6  83  58  39  76
 48   6   2  27  30   7  48
 36  40  12  37   5  24  27
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 70  24   1  88  70  34  20
  0   0   0   0   0   0   0
  2  85  17  51  90  34  55
  0   0   0   0   0   0   0
 97  50  28  46   9  64  21
[torch.DoubleTensor of size 10x7]



In [ ]:
function generateNewSummary(predsummaryT, totalPredsummary, actionvec, k)
    for i=1, predsummaryT:size(1) do
        for j = 1, predsummaryT:size(2) do
            if 
            totalPredsummary[i][{k -predsummaryT:size(2)  , k}]
end

In [ ]:
function stackMemory(newinput, memory_hist, memsize, adapt, use_cuda)
    local sentMemory = torch.cat(newinput[1][1]:double(), memory_hist[1][1]:double(), 1)
    local queryMemory = torch.cat(newinput[1][2]:double(), memory_hist[1][2]:double(), 1)
    local sumryMemory = torch.cat(newinput[1][3]:double(), memory_hist[1][3]:double(), 1)
    local rewardMemory = torch.cat(newinput[2]:double(), memory_hist[2]:double(), 1)

    if adapt thena
        regMemory = torch.cat(newinput[4]:double(), memory_hist[4]:double(), 1)
    end 

    if use_cuda then 
        actionMemory = torch.cat(newinput[3]:double(), memory_hist[3]:double(), 1)
    else 
        actionMemory = torch.cat(newinput[3], memory_hist[3], 1)
    end
    --- specifying rows to index 
    if sentMemory:size(1) <= memsize then
        nend = sentMemory:size(1)
        nstart = 1
    else 
        nstart = math.max(memsize - sentMemory:size(1), 1)
        nend = memsize + nstart
    end
    --- Selecting n last data points
    sentMemory = sentMemory[{{nstart, nend}}]
    queryMemory = queryMemory[{{nstart, nend}}]
    sumryMemory = sumryMemory[{{nstart, nend}}]
    rewardMemory = rewardMemory[{{nstart, nend}}]
    actionMemory = actionMemory[{{nstart, nend}}]

    if use_cuda then
        inputMemory = {sentMemory:cuda(), queryMemory:cuda(), sumryMemory:cuda()}
        rewardMemory = rewardMemory:cuda()
        actionMemory = torch.ByteTensor(#actionMemory):copy(actionMemory):cuda()
    end

    inputMemory = {sentMemory, queryMemory, sumryMemory}
    if adapt then
        regMemory = regMemory[{{nstart, nend}}]
        return {inputMemory, rewardMemory, actionMemory, regMemory}
    end 
    return {inputMemory, rewardMemory, actionMemory}
end    